# 可选实验室 - 正则化成本和梯度

## 目标

在本实验室中，您将：

- 使用正则化项扩展先前的线性和逻辑成本函数。
- 重新运行前面的过度拟合示例，并添加正则化项。

In [11]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from plt_overfit import overfit_example, output
from lab_utils_common import sigmoid

np.set_printoptions(precision=8)

# 添加正则化

<img align="Left" src="../../images/C1_W3_LinearGradientRegularized.png" style=" width:400px; padding: 10px; " >

<img align="Center" src="../../images/C1_W3_LogisticGradientRegularized.png" style=" width:400px; padding: 10px; " >

上面的幻灯片显示了线性回归和逻辑回归的成本函数和梯度函数。笔记：

- 成本
    - 线性回归和逻辑回归之间的成本函数显着不同，但在方程中添加正则化是相同的。
- 坡度
    - 线性回归和逻辑回归的梯度函数非常相似。它们的区别仅在于$f_{wb}$的实现。

## 正则化成本函数

### 正则化线性回归的成本函数

成本函数正则化线性回归的方程为：

$$J(\mathbf{w},b)=\frac{1}{2m}\sum\limits_{i=0}^{m-1}(f_{\mathbf{w},b}(\mathbf{x}^{(i)})-y^{(i)})^2+\frac{\lambda}{2m}\sum_{j=0}^{n-1}w_j^2\tag{1}$$

在哪里：

$$f_{\mathbf{w},b}(\mathbf{x}^{(i)})=\mathbf{w}\cdot\mathbf{x}^{(i)}+b\tag{2}$$

将此与没有正则化的成本函数（您在之前的实验室中实现）进行比较，其形式为：

$$J(\mathbf{w},b)=\frac{1}{2m}\sum\limits_{i=0}^{m-1}(f_{\mathbf{w},b}(\mathbf{x}^{(i)})-y^{(i)})^2$$

区别在于正则化项

<span style="color:blue">
    $\frac{\lambda}{2m}\sum_{j=0}^{n-1}w_j^2$
</span>
    
包含此项会激励梯度下降以最小化参数的大小。请注意，在此示例中，参数$b$未正则化。这是标准做法。

下面是等式（1）和（2）的实现。请注意，这使用了*本课程的标准模式*，即所有`m`示例上的`for loop`。

In [12]:
def compute_cost_linear_reg(X, y, w, b, lambda_=1):
    m = X.shape[0]
    n = len(w)
    cost = 0.
    for i in range(m):
        f_wb_i = np.dot(X[i], w) + b
        cost = cost + (f_wb_i - y[i]) ** 2
    cost = cost / (2 * m)
    reg_cost = 0
    for j in range(n):
        reg_cost += (w[j] ** 2)
    reg_cost = (lambda_ / (2 * m)) * reg_cost
    total_cost = cost + reg_cost
    return total_cost

运行下面的单元格以查看其实际情况。

In [13]:
np.random.seed(1)
X_tmp = np.random.rand(5, 6)
y_tmp = np.array([0, 1, 0, 1, 0])
w_tmp = np.random.rand(X_tmp.shape[1]).reshape(-1, ) - 0.5
b_tmp = 0.5
lambda_tmp = 0.7
cost_tmp = compute_cost_linear_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)
print("Regularized cost:", cost_tmp)

**预期输出**：

<table>
    <tr>
        <td>
            <b>
                Regularized cost: 
            </b>
            0.07917239320214275
        </td>
    </tr>
</table>

### 正则化逻辑回归的成本函数

对于正则化**logistic**回归，成本函数的形式为

$$J(\mathbf{w},b)=\frac{1}{m}\sum_{i=0}^{m-1}\left[-y^{(i)}\log\left(f_{\mathbf{w},b}\left(\mathbf{x}^{(i)}\right)\right)-\left(1-y^{(i)}\right)\log\left(1-f_{\mathbf{w},b}\left(\mathbf{x}^{(i)}\right)\right)\right]+\frac{\lambda}{2m}\sum_{j=0}^{n-1}w_j^2\tag{3}$$

在哪里：

$$f_{\mathbf{w},b}(\mathbf{x}^{(i)})=sigmoid(\mathbf{w}\cdot\mathbf{x}^{(i)}+b)\tag{4}$$

将此与没有正则化的成本函数（您在之前的实验室中实现）进行比较：

$$J(\mathbf{w},b)=\frac{1}{m}\sum_{i=0}^{m-1}\left[-y^{(i)}\log\left(f_{\mathbf{w},b}\left(\mathbf{x}^{(i)}\right)\right)-\left(1-y^{(i)}\right)\log\left(1-f_{\mathbf{w},b}\left(\mathbf{x}^{(i)}\right)\right)\right]$$

与上面线性回归的情况一样，区别在于正则化项，即

<span style="color:blue">
    $\frac{\lambda}{2m}\sum_{j=0}^{n-1}w_j^2$
</span>

包含此项会激励梯度下降以最小化参数的大小。请注意，在此示例中，参数$b$未正则化。这是标准做法。

In [14]:
def compute_cost_logistic_reg(X, y, w, b, lambda_=1):
    m, n = X.shape
    cost = 0.
    for i in range(m):
        z_i = np.dot(X[i], w) + b
        f_wb_i = sigmoid(z_i)
        cost += -y[i] * np.log(f_wb_i) - (1 - y[i]) * np.log(1 - f_wb_i)
    cost = cost / m
    reg_cost = 0
    for j in range(n):
        reg_cost += (w[j] ** 2)
    reg_cost = (lambda_ / (2 * m)) * reg_cost
    total_cost = cost + reg_cost
    return total_cost

运行下面的单元格以查看其实际情况。

In [15]:
np.random.seed(1)
X_tmp = np.random.rand(5, 6)
y_tmp = np.array([0, 1, 0, 1, 0])
w_tmp = np.random.rand(X_tmp.shape[1]).reshape(-1, ) - 0.5
b_tmp = 0.5
lambda_tmp = 0.7
cost_tmp = compute_cost_logistic_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)
print("Regularized cost:", cost_tmp)

**预期输出**：

<table>
    <tr>
        <td>
            <b>
                Regularized cost:
            </b>
        0.6850849138741673
        </td>
    </tr>
</table>

## 正则化梯度下降

运行梯度下降的基本算法不随正则化而改变，它是：

$$\begin{align*}
&\text{repeat until convergence:}\;\lbrace\\
& \;\;\;w_j=w_j-\alpha\frac{\partial J(\mathbf{w},b)}{\partial w_j}\tag{1}\;&\text{for j := 0..n-1}\\ 
& \;\;\;\;\;b=b-\alpha\frac{\partial J(\mathbf{w},b)}{\partial b}\\
&\rbrace
\end{align*}$$

其中每次迭代对所有$j$的$w_j$执行同时更新。

正则化的变化是计算梯度。

### 使用正则化计算梯度（线性/逻辑）

线性回归和逻辑回归的梯度计算几乎相同，仅在$f_{\mathbf{w}b}$的计算上有所不同。

$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}&=\frac{1}{m}\sum\limits_{i=0}^{m-1}(f_{\mathbf{w},b}(\mathbf{x}^{(i)})-y^{(i)})x_{j}^{(i)}+\frac{\lambda}{m}w_j\tag{2}\\
\frac{\partial J(\mathbf{w},b)}{\partial b}&=\frac{1}{m}\sum\limits_{i=0}^{m-1}(f_{\mathbf{w},b}(\mathbf{x}^{(i)})-y^{(i)})\tag{3} 
\end{align*}$$

* m是数据集中训练样本的数量
* $f_{\mathbf{w},b}(x^{(i)})$是模型的预测，而$y^{(i)}$是目标
* 对于<span style="color:blue">**线性**</span>回归模型
  
  $f_{\mathbf{w},b}(x)=\mathbf{w}\cdot\mathbf{x}+b$
* 对于<span style="color:blue">**逻辑**</span>回归模型
  
  $z=\mathbf{w}\cdot\mathbf{x}+b$

  $f_{\mathbf{w},b}(x)=g(z)$

  其中$g(z)$是sigmoid函数：
  
  $g(z)=\frac{1}{1+e^{-z}}$

添加正则化的项是<span style="color:blue">$\frac{\lambda}{m}w_j$</span>。

### 正则化线性回归的梯度函数

In [16]:
def compute_gradient_linear_reg(X, y, w, b, lambda_):
    m, n = X.shape
    dj_dw = np.zeros((n,))
    dj_db = 0.
    for i in range(m):
        err = (np.dot(X[i], w) + b) - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err * X[i, j]
        dj_db = dj_db + err
    dj_dw = dj_dw / m
    dj_db = dj_db / m
    for j in range(n):
        dj_dw[j] = dj_dw[j] + (lambda_ / m) * w[j]
    return dj_db, dj_dw

运行下面的单元格以查看其实际情况。

In [17]:
np.random.seed(1)
X_tmp = np.random.rand(5, 3)
y_tmp = np.array([0, 1, 0, 1, 0])
w_tmp = np.random.rand(X_tmp.shape[1])
b_tmp = 0.5
lambda_tmp = 0.7
dj_db_tmp, dj_dw_tmp = compute_gradient_linear_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)
print(f"dj_db: {dj_db_tmp}", )
print(f"Regularized dj_dw:\n {dj_dw_tmp.tolist()}", )

**期望输出**

```
dj_db: 0.6648774569425726
Regularized dj_dw:
 [0.29653214748822276, 0.4911679625918033, 0.21645877535865857]
 ```

### 正则化逻辑回归的梯度函数

In [18]:
def compute_gradient_logistic_reg(X, y, w, b, lambda_):
    m, n = X.shape
    dj_dw = np.zeros((n,))
    dj_db = 0.0
    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i], w) + b)
        err_i = f_wb_i - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i, j]
        dj_db = dj_db + err_i
    dj_dw = dj_dw / m
    dj_db = dj_db / m
    for j in range(n):
        dj_dw[j] = dj_dw[j] + (lambda_ / m) * w[j]
    return dj_db, dj_dw

运行下面的单元格以查看其实际情况。

In [19]:
np.random.seed(1)
X_tmp = np.random.rand(5, 3)
y_tmp = np.array([0, 1, 0, 1, 0])
w_tmp = np.random.rand(X_tmp.shape[1])
b_tmp = 0.5
lambda_tmp = 0.7
dj_db_tmp, dj_dw_tmp = compute_gradient_logistic_reg(X_tmp, y_tmp, w_tmp, b_tmp, lambda_tmp)
print(f"dj_db: {dj_db_tmp}", )
print(f"Regularized dj_dw:\n {dj_dw_tmp.tolist()}", )

**期望输出**

```
dj_db: 0.341798994972791
Regularized dj_dw:
 [0.17380012933994293, 0.32007507881566943, 0.10776313396851499]
 ```

## 重新运行过拟合示例

In [20]:
plt.close("all")
display(output)
ofit = overfit_example(True)

在上图中，尝试对前面的示例进行正则化。尤其：

- 分类（逻辑回归）
    - 将程度设置为6，将lambda设置为0（无正则化），拟合数据
    - 现在将lambda设置为1（增加正则化），拟合数据，注意差异。
- 回归（线性回归）
    - 尝试相同的过程。

## 恭喜！

你有：

- 为线性和逻辑回归添加回归的成本和梯度例程示例
- 对正则化如何减少过度拟合产生了一些直觉